# 1 The Half-edge Data Structure

In [1]:
from halfedge_mesh.halfedge_mesh import * #import a very lightweight package for half-edge data structures
%load_ext autoreload
%autoreload 2

### 1 Generate mesh for our cube

In [5]:
def create_halfedge_cube():
    ################### 1. Initialise ################################
    cube = HalfedgeMesh()
    cube.update_vertices([
        [-1, -1, -1], [-1, -1, 1], [-1, 1, 1], [-1, 1, -1],
        [1, -1, -1], [1, -1, 1], [1, 1, 1], [1, 1, -1]
    ])
    cube.facets = [Facet(index=i) for i in range(6)]
    cube.halfedges = [Halfedge(index=i) for i in range(24)]
    
    ################## 2. Define Halfedge Connectivity ################  
    # Back face
    cube.halfedges[0].update(vertex=cube.vertices[0], next=cube.halfedges[1], facet=cube.facets[0], opposite=cube.halfedges[10])
    cube.halfedges[1].update(vertex=cube.vertices[4], next=cube.halfedges[2], facet=cube.facets[0], opposite=cube.halfedges[19])
    cube.halfedges[2].update(vertex=cube.vertices[7], next=cube.halfedges[3], facet=cube.facets[0], opposite=cube.halfedges[12])
    cube.halfedges[3].update(vertex=cube.vertices[3], next=cube.halfedges[0], facet=cube.facets[0], opposite=cube.halfedges[21]) 
    # Front face
    cube.halfedges[4].update(vertex=cube.vertices[2], next=cube.halfedges[5], facet=cube.facets[1], opposite=cube.halfedges[14])
    cube.halfedges[5].update(vertex=cube.vertices[6], next=cube.halfedges[6], facet=cube.facets[1], opposite=cube.halfedges[17])
    cube.halfedges[6].update(vertex=cube.vertices[5], next=cube.halfedges[7], facet=cube.facets[1], opposite=cube.halfedges[8])
    cube.halfedges[7].update(vertex=cube.vertices[1], next=cube.halfedges[4], facet=cube.facets[1], opposite=cube.halfedges[23])
    # Bottom face
    cube.halfedges[8].update(vertex=cube.vertices[1], next=cube.halfedges[9], facet=cube.facets[2], opposite=cube.halfedges[6])
    cube.halfedges[9].update(vertex=cube.vertices[5], next=cube.halfedges[10], facet=cube.facets[2], opposite=cube.halfedges[16])
    cube.halfedges[10].update(vertex=cube.vertices[4], next=cube.halfedges[11], facet=cube.facets[2], opposite=cube.halfedges[0])
    cube.halfedges[11].update(vertex=cube.vertices[0], next=cube.halfedges[8], facet=cube.facets[2], opposite=cube.halfedges[20])
    # Top face
    cube.halfedges[12].update(vertex=cube.vertices[3], next=cube.halfedges[13], facet=cube.facets[3], opposite=cube.halfedges[2])
    cube.halfedges[13].update(vertex=cube.vertices[7], next=cube.halfedges[14], facet=cube.facets[3], opposite=cube.halfedges[18])
    cube.halfedges[14].update(vertex=cube.vertices[6], next=cube.halfedges[15], facet=cube.facets[3], opposite=cube.halfedges[4])
    cube.halfedges[15].update(vertex=cube.vertices[2], next=cube.halfedges[12], facet=cube.facets[3], opposite=cube.halfedges[22])
    # Right face
    cube.halfedges[16].update(vertex=cube.vertices[4], next=cube.halfedges[17], facet=cube.facets[4], opposite=cube.halfedges[9])
    cube.halfedges[17].update(vertex=cube.vertices[5], next=cube.halfedges[18], facet=cube.facets[4], opposite=cube.halfedges[5])
    cube.halfedges[18].update(vertex=cube.vertices[6], next=cube.halfedges[19], facet=cube.facets[4], opposite=cube.halfedges[13])
    cube.halfedges[19].update(vertex=cube.vertices[7], next=cube.halfedges[16], facet=cube.facets[4], opposite=cube.halfedges[1])
    # Left face
    cube.halfedges[20].update(vertex=cube.vertices[1], next=cube.halfedges[21], facet=cube.facets[5], opposite=cube.halfedges[11])
    cube.halfedges[21].update(vertex=cube.vertices[0], next=cube.halfedges[22], facet=cube.facets[5], opposite=cube.halfedges[3])
    cube.halfedges[22].update(vertex=cube.vertices[3], next=cube.halfedges[23], facet=cube.facets[5], opposite=cube.halfedges[15])
    cube.halfedges[23].update(vertex=cube.vertices[2], next=cube.halfedges[20], facet=cube.facets[5], opposite=cube.halfedges[7])

    ########## 3. Define Facet Connectivity ##############
    cube.facets[0].update(halfedge=cube.halfedges[0])
    cube.facets[1].update(halfedge=cube.halfedges[4])
    cube.facets[2].update(halfedge=cube.halfedges[8])
    cube.facets[3].update(halfedge=cube.halfedges[12])
    cube.facets[4].update(halfedge=cube.halfedges[16])
    cube.facets[5].update(halfedge=cube.halfedges[20])
    
    ########## 4. Define Vertex Connectivity ##############
    cube.vertices[0].update(halfedge=cube.halfedges[0])
    cube.vertices[1].update(halfedge=cube.halfedges[7])
    cube.vertices[2].update(halfedge=cube.halfedges[15])
    cube.vertices[3].update(halfedge=cube.halfedges[12])
    cube.vertices[4].update(halfedge=cube.halfedges[1])
    cube.vertices[5].update(halfedge=cube.halfedges[17])
    cube.vertices[6].update(halfedge=cube.halfedges[5])
    cube.vertices[7].update(halfedge=cube.halfedges[19])
    
    return cube

cube = create_halfedge_cube()
cube.flip()
cube.write_off('my_cube.off') #This converts the halfedge triangle into a .off file. Open it in Meshlab.

### 2

In [17]:
def calculate_centroid(face):
    """计算面的质心"""
    vertices = face.get_vertices()
    x = sum(vertex.x for vertex in vertices) / len(vertices)
    y = sum(vertex.y for vertex in vertices) / len(vertices)
    z = sum(vertex.z for vertex in vertices) / len(vertices)
    return [x, y, z]

## 解释一下facet.get_vertices()
## 就是先找到面对应的那个halfedge，然后通过不断调用next halfedge找到面所有的halfedge的起点，也就找到了面的所有vertices



### 3

In [18]:
def create_halfedge_tetrahedron():
    ################### 1. Initialise ################################
    
    tetra = HalfedgeMesh() #Create a halfedge-mesh object
    tetra.update_vertices([ [1,0,-2**(-0.5)], [-1,0,-2**(-0.5)], [0,1,2**(-0.5)] , [0,-1,2**(-0.5)]]) #Create vertices
    tetra.facets = [ Facet(index=i) for i in range(4) ] 
    tetra.halfedges = [ Halfedge(index = i) for i in range(12) ] 
    
    ################## 2. Define Halfedge Connectivity ################
    
    tetra.halfedges[0].update ( vertex = tetra.vertices[0], next = tetra.halfedges[1] , facet = tetra.facets[0] ,
                               opposite = tetra.halfedges[3] )
    
    tetra.halfedges[1].update ( vertex =  tetra.vertices[1], next = tetra.halfedges[2], facet = tetra.facets[0] ,
                               opposite = tetra.halfedges[5])
    
    tetra.halfedges[2].update ( vertex =  tetra.vertices[2], next = tetra.halfedges[0], facet = tetra.facets[0] ,
                              opposite = tetra.halfedges[4]) 
    
    tetra.halfedges[3].update ( vertex = tetra.vertices[1], next = tetra.halfedges[7], facet = tetra.facets[3] ,
                              opposite = tetra.halfedges[0])
    
    tetra.halfedges[4].update ( vertex = tetra.vertices[0], next = tetra.halfedges[6], facet = tetra.facets[1], 
                               opposite = tetra.halfedges[2] ) 
    
    tetra.halfedges[5].update ( vertex = tetra.vertices[2], next = tetra.halfedges[9], facet = tetra.facets[2],
                               opposite =  tetra.halfedges[1])
    
    tetra.halfedges[6].update ( vertex = tetra.vertices[2], next = tetra.halfedges[11] , facet = tetra.facets[1],
                               opposite =  tetra.halfedges[10])
    
    tetra.halfedges[7].update ( vertex = tetra.vertices[0], next = tetra.halfedges[8], facet = tetra.facets[3],
                               opposite =  tetra.halfedges[11])
    
    tetra.halfedges[8].update ( vertex = tetra.vertices[3], next = tetra.halfedges[3], facet = tetra.facets[3],
                               opposite = tetra.halfedges[9] )

    tetra.halfedges[9].update ( vertex = tetra.vertices[1], next = tetra.halfedges[10], facet = tetra.facets[2],
                               opposite = tetra.halfedges[8] )
    
    tetra.halfedges[10].update ( vertex = tetra.vertices[3], next = tetra.halfedges[5], facet = tetra.facets[2],
                                opposite =  tetra.halfedges[6])
    
    tetra.halfedges[11].update ( vertex = tetra.vertices[3], next = tetra.halfedges[4], facet = tetra.facets[1],
                                opposite = tetra.halfedges[7] )
    
    
    ########## 3. Define Facet Connectivity ##############
    
    tetra.facets[0].update (halfedge = tetra.halfedges[0])
    tetra.facets[1].update (halfedge = tetra.halfedges[6])
    tetra.facets[2].update (halfedge = tetra.halfedges[10])
    tetra.facets[3].update (halfedge = tetra.halfedges[7])

    ########## 4. Define Vertex Connectivity ##############

    tetra.vertices[0].update (halfedge = tetra.halfedges[0])
    tetra.vertices[1].update (halfedge = tetra.halfedges[1])
    tetra.vertices[2].update (halfedge = tetra.halfedges[2])
    tetra.vertices[3].update (halfedge = tetra.halfedges[8])

    return tetra

tetra = create_halfedge_tetrahedron()
tetra.write_off('my_tetrahedron.off') 

In [55]:
import numpy as np
import math

def calculate_original_mesh_centroid(original_mesh):
    # 初始化累加器
    sum_x = 0
    sum_y = 0
    sum_z = 0

    # 遍历所有顶点，累加它们的坐标
    for vertex in original_mesh.vertices:
        sum_x += vertex.x
        sum_y += vertex.y
        sum_z += vertex.z

    # 计算顶点的总数
    num_vertices = len(original_mesh.vertices)

    # 如果顶点总数大于0，则计算平均坐标
    if num_vertices > 0:
        centroid_x = sum_x / num_vertices
        centroid_y = sum_y / num_vertices
        centroid_z = sum_z / num_vertices
        # print(centroid_x, centroid_y, centroid_z)
        return np.array([centroid_x, centroid_y, centroid_z])
    else:
        # 如果网格中没有顶点，则返回原点作为质心
        return np.array([0, 0, 0])


def correct_halfedges_order(dual_facet, original_mesh_centroid):
    # 计算对偶面的质心
    dual_facet_centroid = calculate_centroid(dual_facet)

    # 获取对偶面的所有半边
    halfedges = []
    start_he = dual_facet.halfedge
    he = start_he
    while True:
        halfedges.append(he)
        he = he.next
        if he == start_he:
            break

    # 计算从对偶面质心到原网格质心的向量
    to_original_centroid_vector = original_mesh_centroid - dual_facet_centroid

    # 计算第一个半边的法线和向量的点乘
    first_he_normal = dual_facet.get_normal()
    dot_product = np.dot(first_he_normal, to_original_centroid_vector)

    # 如果点乘小于0，反转半边的顺序
    if dot_product > 0:
        halfedges.reverse()

        # 重新设置每个半边的next和prev属性
        for i, he in enumerate(halfedges):
            next_index = (i + 1) % len(halfedges)
            prev_index = (i - 1 + len(halfedges)) % len(halfedges)
            he.next = halfedges[next_index]
            he.prev = halfedges[prev_index]

        # 更新对偶面的halfedge为反转后的第一个半边
        dual_facet.halfedge = halfedges[0]


def compute_dual_mesh(original_mesh):
    dual_vertices = []  # 对偶图的顶点列表
    dual_halfedges = []  # 对偶图的半边列表
    dual_facets = []  # 对偶图的面列表

    # 为原始网格的每个面创建一个对偶顶点
    for facet in original_mesh.facets:
        center = calculate_centroid(facet)  # 计算质心
        dual_vertex = Vertex(center[0], center[1], center[2], len(dual_vertices))
        dual_vertices.append(dual_vertex)
        
    
    # 为原始网格的每个顶点创建一个对偶面，并创建对偶半边
    for vertex in original_mesh.vertices:
        he_map = {}
        new_facet = Facet(index=vertex.index)  # 创建对偶面
        dual_facets.append(new_facet)

        # 找到以该顶点为起点的所有半边
        start_halfedges = [he for he in original_mesh.halfedges if he.vertex == vertex]
        first_dual_he = None  # 用于闭合循环，连接最后一个和第一个对偶半边

        # 为每个半边创建对偶半边，并设置其next和prev关系
        for i, he in enumerate(start_halfedges):
            if he.facet and he.opposite and he.opposite.facet:
                dual_start_vertex = dual_vertices[he.facet.index]
                dual_end_vertex = dual_vertices[he.opposite.facet.index]

                dual_he = Halfedge(vertex=dual_start_vertex, facet=new_facet)
                dual_halfedges.append(dual_he)
                # 将起始顶点和结束顶点的组合映射到对偶半边
                he_map[(dual_start_vertex.index, dual_end_vertex.index)] = dual_he

                # 为对偶顶点设置halfedge属性
                if not dual_start_vertex.halfedge:
                    dual_start_vertex.halfedge = dual_he
                if not new_facet.halfedge:
                    new_facet.halfedge = dual_he  # 设置对偶面的第一个半边
        # print(len(dual_halfedges))
        # 遍历映射表，设置每个半边的next和prev属性
        for (start_idx, end_idx), he in he_map.items():
            # print((start_idx, end_idx))
            # 查找所有可能的候选半边，它们以当前半边的结束顶点为起始顶点
            candidates = [he_candidate for (s_idx, _), he_candidate in he_map.items() if s_idx == end_idx]

            # 从候选中选择一个作为next_he
            # 这里的选择逻辑可能需要根据你的具体情况来定
            next_he = True
            for candidate in candidates:
                next_he = False
                he.next = candidate
                candidate.prev = he
                break
            if next_he :
                print("wtf")

            
        # 修正为顺时针方向
        correct_halfedges_order(new_facet, calculate_original_mesh_centroid(original_mesh))
        # print("123")


    # 为每个对偶半边设置opposite属性
    for he in dual_halfedges:
        for candidate in dual_halfedges:
            if he.vertex == candidate.next.vertex and he.next.vertex == candidate.vertex:
                he.opposite = candidate
                candidate.opposite = he
                break

    dual_mesh = HalfedgeMesh(vertices=dual_vertices, halfedges=dual_halfedges, facets=dual_facets)
    
    return dual_mesh



# 试了下tetra怎么再对偶都没问题呀
dual_cube = compute_dual_mesh(cube)
dual_cube.write_off('my_dual_cube1.off')
dual_cube1 = compute_dual_mesh(dual_cube)
print(len(dual_cube1.facets))
print("asd")
# 这里对偶再对偶，就不对了，看了下是面的输出不是按顺序来而是开始交错了，说明沿着he往下找的时候开始出错了
# 现在对了
dual_cube1.write_off('my_dual_cube2.off') #This converts the halfedge triangle into a .off file. Open it in Meshlab.


6
asd


## Task 2: ICP

### 2.1

In [2]:
import sys,os

RES_PATH = 'bunny_v2'
if not os.path.exists(RES_PATH):
    print( 'cannot find /resources, please update RES_PATH')
    exit(1)
else:
    print('found resources')

import pyglet
pyglet.options['shadow_window'] = False
import pyrender
import numpy as np
import trimesh
import matplotlib
import matplotlib.pyplot as plt
import open3d as o3d

found resources


d:\Anaconda\envs\godzilla\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [70]:
import time
from scipy.spatial import KDTree
from scipy.sparse import diags
from scipy.sparse.linalg import svds

RES_PATH = 'bunny_v2'  # 请替换为实际路径

mesh_src = os.path.join(RES_PATH, 'bun000_v2.ply')
assert os.path.exists(mesh_src), 'cannot found:' + mesh_src
mesh1 = o3d.io.read_triangle_mesh(mesh_src)

mesh_dst = os.path.join(RES_PATH, 'bun045_v2.ply')
assert os.path.exists(mesh_dst), 'cannot found:' + mesh_dst
mesh2 = o3d.io.read_triangle_mesh(mesh_dst)

points_A = np.asarray(mesh1.vertices)
points_B = np.asarray(mesh2.vertices)

def visualize_iteration(src, dst, threshold=1e-2):
    # 将 numpy 数组转换为 Open3D 点云
    src_pcd = o3d.geometry.PointCloud()
    src_pcd.points = o3d.utility.Vector3dVector(src)
    src_pcd.paint_uniform_color([1, 0, 0])  # 红色表示 meshA

    dst_pcd = o3d.geometry.PointCloud()
    dst_pcd.points = o3d.utility.Vector3dVector(dst)
    dst_pcd.paint_uniform_color([0, 0, 1])  # 蓝色表示 meshB

    # 创建 KD 树来查找最近点对
    src_tree = KDTree(src)
    dst_tree = KDTree(dst)

    # 查找两个点云中每个点的最近邻居并计算距离
    distances_src_to_dst, indices_src_to_dst = src_tree.query(dst)
    distances_dst_to_src, indices_dst_to_src = dst_tree.query(src)

    # 根据距离阈值标记重叠区域的点为绿色
    for i, distance in enumerate(distances_src_to_dst):
        if distance < threshold:  # 如果距离小于阈值，则视为重叠区域
            dst_pcd.colors[i] = [0, 1, 0]  # 绿色

    for i, distance in enumerate(distances_dst_to_src):
        if distance < threshold:
            src_pcd.colors[i] = [0, 1, 0]  # 绿色

    # 可视化点云
    o3d.visualization.draw_geometries([src_pcd, dst_pcd])


def best_fit_transform(A, B, weights=None):
    assert len(A) == len(B)

    if weights is None:
        weights = np.ones(len(A))

    centroid_A = np.average(A, axis=0, weights=weights)
    centroid_B = np.average(B, axis=0, weights=weights)

    AA = A - centroid_A
    BB = B - centroid_B

    # 直接应用权重，避免创建大的对角矩阵
    AA_weighted = AA * weights[:, np.newaxis]
    H = AA_weighted.T @ BB

    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T
    if np.linalg.det(R) < 0:
        Vt[2, :] *= -1
        R = Vt.T @ U.T
    t = centroid_B.T - R @ centroid_A.T
    t = t.reshape(-1, 1)
    return R, t

def icp(src_cp, dst_cp, max_iterations=100, tolerance=0.00005):
    # 这个tolerance之前是0.001，我把它调整的0.00005 / 0.0001这么小之后，显著改进了结果
    # 感觉rotate修正问题中，这个一定要调整的特别小，当我试图用0.001的时候，我一度以为我写错了，改成0.00005才正常全重叠
    # 我感觉这里可以加一个条件，如果dis总和小于多少也停止
    start_time = time.time()
    src = np.copy(src_cp)
    dst = np.copy(dst_cp)
    prev_error = np.inf
    tree = KDTree(dst)

    for i in range(max_iterations):
        # 在dst建的树中，查询最近的几个点
        distances, indices = tree.query(src, k = 1)
        # 越近权重越大？说明越信任这个点
        weights = 1.0 / (distances + 1e-6)
        T = best_fit_transform(src, dst[indices], weights=weights)
        src = (T[0] @ src.T + T[1]).T
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error
        # 可视化每次迭代的对齐效果（可选）
        # visualize_iteration(src, dst)

    T = best_fit_transform(src_cp, src, weights=weights)
    total_time = time.time() - start_time
    return T[0], T[1], i, total_time 


# 这里不对模型做任何改动，只是求变换
R, t, iterations, time_total = icp(points_B, points_A)
# 应用ICP变换到第二个网格
mesh2.transform(np.vstack((np.hstack((R, t)), [0, 0, 0, 1])))

visualize_iteration(points_A, points_B)

# 可视化对齐后的网格
o3d.visualization.draw_geometries([mesh1, mesh2])
aligned_mesh_path = os.path.join(RES_PATH, 'bun045_v2_aligned.ply')
combined_mesh = mesh1 + mesh2  # 合并网格
o3d.io.write_triangle_mesh(aligned_mesh_path, combined_mesh)  # 保存合并后的网格


True

### 2.2

In [68]:
def calculate_mean_error(A_mesh, B_mesh, R, t):
    # 提取顶点作为点云
    A_points = np.asarray(A_mesh.vertices)
    B_points = np.asarray(B_mesh.vertices)

    # 变换A点云
    transformed_A = np.dot(R, A_points.T).T + t.T

    # 构建B点云的KD树
    tree = KDTree(B_points)

    # 查找最近点并计算距离
    distances, _ = tree.query(transformed_A)

    # 计算平均误差
    mean_error = np.mean(distances)
    return mean_error

def clone_mesh(mesh):
    # 创建一个新的 TriangleMesh 对象
    cloned_mesh = o3d.geometry.TriangleMesh()
    
    # 将原网格的顶点和面复制给新网格
    cloned_mesh.vertices = o3d.utility.Vector3dVector(np.asarray(mesh.vertices))
    cloned_mesh.triangles = o3d.utility.Vector3iVector(np.asarray(mesh.triangles))
    
    # 如果原网格有顶点颜色和法线，也复制它们
    if mesh.has_vertex_colors():
        cloned_mesh.vertex_colors = o3d.utility.Vector3dVector(np.asarray(mesh.vertex_colors))
    if mesh.has_vertex_normals():
        cloned_mesh.vertex_normals = o3d.utility.Vector3dVector(np.asarray(mesh.vertex_normals))
    
    return cloned_mesh


In [72]:
import copy

def rotate_mesh(mesh, angle, axis='z'):
    angle_rad = np.radians(angle)
    copy_mesh = clone_mesh(mesh)
    
    # 根据角度和轴生成旋转矩阵
    if axis == 'z':
        R = o3d.geometry.get_rotation_matrix_from_axis_angle([0, 0, angle_rad])
    elif axis == 'y':
        R = o3d.geometry.get_rotation_matrix_from_axis_angle([0, angle_rad , 0])
    elif axis == 'x':
        R = o3d.geometry.get_rotation_matrix_from_axis_angle([angle_rad, 0, 0])
    else:
        raise ValueError("Axis must be 'x', 'y', or 'z'")
    # 应用旋转
    copy_mesh.rotate(R, (0, 0, 0))
    return copy_mesh

def add_noise(mesh, noise_level):
    copy_mesh = clone_mesh(mesh)
    # 获取网格的边界盒尺寸
    bounding_box = copy_mesh.get_axis_aligned_bounding_box()
    extent = bounding_box.get_extent()
    max_extent = np.max(extent)
    
    # 计算噪声标准差
    sigma = noise_level * max_extent
    # 向每个顶点添加高斯噪声
    vertices = np.asarray(copy_mesh.vertices)
    noise = np.random.normal(0, sigma, vertices.shape)
    copy_mesh.vertices = o3d.utility.Vector3dVector(vertices + noise)
    return copy_mesh


# 评估旋转扰动
def evaluate_rotation(mesh, angles):
    rotation_results = []
    
    for angle in angles:
        # 应用旋转扰动
        rotated_mesh = rotate_mesh(mesh, angle)
        rotated_points = np.asarray(rotated_mesh.vertices)
        points =  np.asarray(mesh.vertices)
        
        # 使用ICP算法对齐
        R, t, iterations, total_time = icp(rotated_points, points)
        rotated_mesh.transform(np.vstack((np.hstack((R, t)), [0, 0, 0, 1])))
        visualize_iteration(np.asarray(rotated_mesh.vertices), np.asarray(mesh.vertices))
        
        aligned_mesh_path = os.path.join(RES_PATH, 'zxc.ply')
        combined_mesh = rotated_mesh + mesh  # 合并网格
        o3d.io.write_triangle_mesh(aligned_mesh_path, combined_mesh)  # 保存合并后的网格

        # 评估对齐准确性   
        mean_error = calculate_mean_error(rotated_mesh, mesh, R, t)
        print(f"Angle: {angle} degrees, Mean error: {mean_error}, Iterations: {iterations}, time:{total_time}")
        rotation_results.append((angle, mean_error, iterations, total_time))
        
        del rotated_mesh # 释放内存
        
    return rotation_results


# 评估添加噪声
def evaluate_noise(mesh, noise_levels):
    noise_results = []
    
    for noise_level in noise_levels:
        # 添加噪声扰动
        noisy_mesh = add_noise(mesh, noise_level)
        points =  np.asarray(mesh.vertices)
        noisy_points = np.asarray(noisy_mesh.vertices)
        visualize_iteration(np.asarray(noisy_mesh.vertices), np.asarray(mesh.vertices))
        
        # 使用ICP算法对齐
        R, t, iterations, total_time = icp(noisy_points, points)
        noisy_mesh.transform(np.vstack((np.hstack((R, t)), [0, 0, 0, 1])))
        visualize_iteration(np.asarray(noisy_mesh.vertices), np.asarray(mesh.vertices))
        
        # 评估对齐准确性
        mean_error = calculate_mean_error(noisy_mesh, mesh, R, t)
        print(f"Noise level: {noise_level}, Mean error: {mean_error}, Iterations: {iterations}, time:{total_time}")
        noise_results.append((noise_level, mean_error, iterations, total_time))
        visualize_iteration(np.asarray(noisy_mesh.vertices), np.asarray(mesh.vertices))
    return noise_results


# 示例角度和噪声级别
angles = [0, 5, 10, 15, 20, 25, 30]
noise_levels = [0.0, 0.01, 0.02, 0.03, 0.04, 0.05]

# 调用评估函数
rotation_data = evaluate_rotation(mesh1, angles)
noise_data = evaluate_noise(mesh1, noise_levels)

# 解包评估数据
angles_evaluated, mean_errors_rotation, iterations_rotation, times_rotation = zip(*rotation_data)
noise_evaluated, mean_errors_noise, iterations_noise, times_noise = zip(*noise_data)

# 绘制旋转扰动结果
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(angles_evaluated, mean_errors_rotation, 'o-', label='Mean Error')
plt.plot(angles_evaluated, times_rotation, 's-', label='Time')
plt.xlabel('Rotation Angle (degrees)')
plt.ylabel('Mean Error / Time (seconds)')
plt.title('ICP Performance on Rotation Perturbation')
plt.legend()

# 绘制噪声扰动结果
plt.subplot(1, 2, 2)
plt.plot(noise_evaluated, mean_errors_noise, 'o-', label='Mean Error')
plt.plot(noise_evaluated, times_noise, 's-', label='Time')
plt.xlabel('Noise Level')
plt.ylabel('Mean Error / Time (seconds)')
plt.title('ICP Performance on Noise Perturbation')
plt.legend()

plt.tight_layout()
plt.show()

Angle: 0 degrees, Mean error: 3.6930817724182594e-17, Iterations: 1, time:0.5443577766418457


KeyboardInterrupt: 